#**AI ACCOUNTING CHAPTER 2: REASONERS**

---

##0.REFERENCE

https://claude.ai/share/01696c05-ee2f-4a51-9765-836a503cc0fb

##1.CONTEXT

**Introduction: Building Trustworthy AI Systems for Professional Accounting and Audit**

**Why This Notebook Matters**

If you've used ChatGPT, Claude, or other AI chatbots for work, you've likely experienced both their remarkable capabilities and their concerning limitations. Perhaps you asked an AI to help draft a client memo and it sounded authoritative but cited nonexistent accounting standards. Maybe you used it to analyze financial data and couldn't explain to your supervisor exactly what assumptions the AI made. Or you simply felt uneasy about how easily these tools generate professional-sounding content without any documentation trail that would survive a quality control review.

This notebook addresses a critical gap between casual AI experimentation and professional-grade AI deployment. While consumer chatbots are designed for convenience and general helpfulness, professional services—especially accounting, auditing, and tax work—demand something fundamentally different: auditability, reproducibility, and explicit risk management. The stakes are simply too high to treat AI as a magical black box that produces outputs we hope are correct.

**The Professional Services Dilemma**

Accounting and audit professionals face unique constraints that casual AI users don't encounter. When you sign off on financial statements, tax returns, or internal control assessments, you're accepting legal and ethical responsibility for that work. Regulators, courts, and professional standards bodies expect you to document your procedures, demonstrate professional skepticism, and maintain audit trails that could be scrutinized years later. None of the popular consumer AI chatbots were designed with these requirements in mind.

Consider what happens in a typical ChatGPT conversation about a technical accounting issue. You describe a scenario, the AI provides an analysis citing ASC standards and offering conclusions about proper treatment. The response sounds confident and well-reasoned. But can you verify those citations are real? Can you reproduce the exact same analysis six months from now? Can you show a reviewer what facts you provided versus what the AI assumed? Can you demonstrate that you exercised professional skepticism by considering alternative interpretations? The answer to all these questions is usually no.

This creates a dangerous paradox: AI tools are powerful enough to meaningfully assist professional work, but not trustworthy enough to rely upon without extensive safeguards. The solution isn't to ban AI from professional practice—that ship has sailed, and the productivity gains are too substantial to ignore. Instead, we need to engineer AI systems that respect the governance requirements of professional services from the ground up.

**What This Notebook Teaches: Level 2 Reasoning with Full Governance**

This notebook introduces you to "Level 2 Reasoners"—a specific approach to AI assistance that provides structured reasoning support while maintaining strict boundaries about what the AI can and cannot do. Unlike Level 1 systems (simple question-answering) or Level 3 systems (autonomous agents that execute multi-step tasks), Level 2 focuses on helping you structure your thinking without replacing your professional judgment.

The architecture implements seven interconnected governance mechanisms that transform casual AI chat into auditable professional assistance:

**Strict Input/Output Contracts:** Every AI interaction uses a mandatory JSON schema with eight required fields—task description, facts provided, assumptions made, open questions, reasoning analysis, risk flags, draft output, and verification status. This structure forces both you and the AI to be explicit about what's known versus inferred versus missing.

**Facts Versus Assumptions Discipline:** The system distinguishes rigorously between facts you explicitly stated and assumptions the AI inferred. This seemingly simple distinction is transformative—it prevents the common failure mode where AI confidently fills knowledge gaps with plausible-sounding fabrications, and you don't realize where your facts ended and the AI's guesswork began.

**Confidentiality Protection:** Before any text reaches the AI, automated redaction utilities scan for personally identifiable information—emails, phone numbers, SSNs, addresses. While not perfect, this creates a safety net against accidental disclosure of client data. The system also implements data minimization principles, sending only the facts necessary for the reasoning task rather than entire documents.

**"Not Verified" Discipline:** Whenever the AI references authoritative sources (ASC standards, PCAOB requirements, IRS regulations), the output is automatically flagged "Not verified" and includes a checklist of specific items to verify in the actual source materials. This prevents the dangerous practice of treating AI citations as reliable without independent confirmation.

**Professional Skepticism by Design:** The system requires every analysis to include "disconfirming evidence questions"—specific inquiries that would challenge or falsify the working hypotheses. This operationalizes the professional skepticism standard that auditing standards require but that many practitioners struggle to apply consistently.

**Complete Audit Trail:** Every run generates five artifact categories—a run manifest with configuration details and cryptographic hashes, a prompts log with redacted interactions, a risk register documenting flagged issues, structured deliverables with human-readable summaries, and a final zip bundle containing the complete audit trail. Six months later, you can prove exactly what you asked, what the AI said, what risks were identified, and what configuration was used.

**Automated Risk Flagging:** The system doesn't trust the AI to police itself. After each response, automated checks scan for missing open questions (suggesting overconfidence), unauthorized authority citations (suggesting hallucination), and conclusion-like language in sections that should contain reasoning notes only (suggesting boundary violations). Risks are categorized by type and severity, creating a structured register rather than vague unease.

**The Notebook Structure: Ten Cells from Theory to Practice**

The notebook follows a carefully designed progression across ten cells. Cell 1 orients you to the governance-first philosophy and explains what Level 2 reasoning means in plain English. Cells 2-3 handle the technical setup—installing libraries, connecting to Claude's API, and configuring the model parameters that control response behavior.

Cells 4-5 build the governance infrastructure—the logging utilities, cryptographic hashing functions, redaction tools, and data minimization helpers that run invisibly in the background to ensure every interaction is documented and protected. Cell 6 implements the core "reasoner wrapper" that enforces the strict JSON schema, adds automated risk checks, and maintains the audit trail.

Cell 7 defines four realistic mini-cases spanning financial statement audit, SOX/ICFR compliance, tax research, and training methodology. These aren't toy examples—they're modeled on actual professional scenarios with incomplete information, ambiguous facts, and judgment calls required. Cell 8 runs all four demos, generating structured outputs and a summary comparison table. Cell 9 turns the notebook interactive, letting you input your own scenario and receive the same governed reasoning support. Cell 10 packages everything into a downloadable audit trail with a comprehensive README explaining how to review and reproduce the work.

**Key Lessons: What You Must Take Away**

First, AI capability and AI risk scale together—more powerful models require stronger controls, not weaker ones. The governance mechanisms in this notebook aren't obstacles to productivity; they're what make professional AI use possible at all.

Second, the distinction between facts and assumptions is foundational to trustworthy AI assistance. Every time you interact with AI, ask yourself: what did I explicitly state, what did the AI infer, and what remains uncertain? If you can't answer clearly, you're not in control of the process.

Third, verification is non-negotiable. "Not verified" isn't a bug or a limitation—it's an accurate description of AI's relationship to authoritative sources. The AI can help you frame research questions and structure your analysis, but you must verify every citation, standard, or requirement yourself.

Fourth, reproducibility requires deliberate engineering. The configuration hash, environment fingerprint, and cryptographic hashes aren't academic exercises—they're what separate casual experimentation from professional work product that could be defended in litigation or regulatory examination.

Fifth, professional skepticism must be operationalized, not just preached. The requirement to generate disconfirming evidence questions transforms skepticism from an aspirational standard into a concrete deliverable that reviewers can evaluate.

Finally, AI is not a substitute for professional judgment—it's a tool for structuring that judgment more systematically. The goal isn't to let AI make decisions for you; it's to make your own decision-making process more rigorous, transparent, and defensible.

This notebook represents one possible answer to the question: how do we harness AI's analytical power while respecting the governance requirements of professional services? The specific implementation details will evolve as models improve and standards develop, but the underlying principles—explicit boundaries, rigorous documentation, facts versus assumptions, verification discipline, and audit trails—will remain essential to any trustworthy professional AI system.

##2.LIBRARIES AND ENVIRONMENT

In [1]:
# Cell 2: Install Dependencies + Imports + Run Directory Setup

print("=" * 70)
print("CELL 2: Installing dependencies and setting up run environment")
print("=" * 70)

# Install anthropic library
!pip install -q anthropic

# Standard library imports
import json
import os
import re
import hashlib
import platform
import textwrap
from datetime import datetime
from pathlib import Path
import subprocess

print("\n✓ Dependencies installed")
print("✓ Imports loaded: json, os, re, hashlib, platform, datetime, pathlib, subprocess")

# Create run directory with timestamp
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_BASE_DIR = Path("/content/ai_audit_ch2_runs")
RUN_DIR = RUN_BASE_DIR / f"run_{timestamp_str}"
DELIVERABLES_DIR = RUN_DIR / "deliverables"

# Create directories
RUN_DIR.mkdir(parents=True, exist_ok=True)
DELIVERABLES_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Run directory created: {RUN_DIR}")
print(f"✓ Deliverables directory: {DELIVERABLES_DIR}")
print("\nReady to proceed to API key setup (Cell 3)")

CELL 2: Installing dependencies and setting up run environment
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 8.1 MB/s eta 0:00:00

✓ Dependencies installed
✓ Imports loaded: json, os, re, hashlib, platform, datetime, pathlib, subprocess

✓ Run directory created: /content/ai_audit_ch2_runs/run_20260111_192247
✓ Deliverables directory: /content/ai_audit_ch2_runs/run_20260111_192247/deliverables

Ready to proceed to API key setup (Cell 3)


##3.API CALL AND CLAUDE CLIENT INITIALIZATION

###3.1.OVERVIEW

**Cell 3: Setting Up Your Connection to Claude**

**What This Cell Does:**

This cell establishes the vital connection between your Google Colab notebook and Anthropic's Claude AI system. Think of it as plugging in a power cord before you can use an appliance—without this connection, none of the AI reasoning capabilities will work.

**Why API Keys Matter:**

An API key is like a secure password that allows your notebook to communicate with Claude's servers. Anthropic provides these keys to verified users, and they come with usage limits and security protections. You store this key in Colab's "Secrets" feature (found in the left sidebar with a key icon 🔑), which keeps it encrypted and hidden from anyone who might view your notebook later.

**The Three Critical Settings:**

Once the key is loaded, this cell configures three technical parameters that control how Claude responds:

**Model Selection:** We use "claude-sonnet-4-5-20250929," which is Anthropic's most advanced reasoning model as of this training program. Different models have different strengths—this one excels at structured analytical thinking, which is exactly what CPAs and auditors need.

**Temperature (0.2):** This setting controls randomness in Claude's responses. Temperature ranges from 0 to 1, where 0 means completely deterministic (same input always produces same output) and 1 means highly creative and varied. We set it to 0.2 because accounting and audit work demands consistency and reliability. You want Claude to structure your thinking the same way each time, not give you wildly different hypotheses for the same set of facts.

**Max Tokens (1,200):** A "token" is roughly equivalent to a word or word fragment. By limiting responses to 1,200 tokens, we ensure Claude provides focused, actionable outputs rather than lengthy essays. This keeps the reasoning support practical and forces the model to prioritize the most important insights.

**What Success Looks Like:**

When this cell runs successfully, you'll see confirmation messages showing that your API key loaded correctly and displaying your model configuration. If something goes wrong, the cell provides clear instructions on how to add your API key to Colab's secrets. Without this setup, the rest of the notebook cannot function, so take time to get this right before proceeding.

###3.2.CODE AND IMPLEMENTATION

In [18]:
# Cell 3: API Key + Client Initialization

print("=" * 70)
print("CELL 3: API Key and Anthropic Client Initialization")
print("=" * 70)

import anthropic
from google.colab import userdata

# Retrieve API key from Colab secrets
try:
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
    api_key_loaded = True
    print("\n✓ API key loaded: YES")
except Exception as e:
    api_key_loaded = False
    print(f"\n✗ API key loaded: NO")
    print(f"Error: {e}")
    print("\nTo fix: Go to Colab menu → Secrets (🔑) → Add 'ANTHROPIC_API_KEY'")

if api_key_loaded:
    # Initialize Anthropic client
    client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

    # Model configuration - HIGH token limit for detailed accounting scenarios
    MODEL = "claude-sonnet-4-5-20250929"
    TEMPERATURE = 0.2
    MAX_TOKENS = 8000  # Increased to 8000 for complex multi-section outputs

    print(f"\n✓ Anthropic client initialized")
    print(f"✓ Model: {MODEL}")
    print(f"✓ Temperature: {TEMPERATURE} (low for deterministic reasoning)")
    print(f"✓ Max tokens: {MAX_TOKENS} (high limit for detailed analysis)")
    print("\nReady to proceed to governance utilities (Cell 4)")
else:
    print("\n⚠ Cannot proceed without API key. Please add it to Colab secrets.")

CELL 3: API Key and Anthropic Client Initialization

✓ API key loaded: YES

✓ Anthropic client initialized
✓ Model: claude-sonnet-4-5-20250929
✓ Temperature: 0.2 (low for deterministic reasoning)
✓ Max tokens: 8000 (high limit for detailed analysis)

Ready to proceed to governance utilities (Cell 4)


##4.GOVERNANCE UTILITIES

###4.1.OVERVIEW

**Cell 4: Building the Governance Foundation – Your Audit Trail Begins Here**

**What This Cell Does:**

This cell creates the complete infrastructure for tracking and documenting every action the AI takes during your session. Think of it as setting up a flight recorder in an airplane—it captures everything that happens so you can review, audit, and reproduce the work later. In professional accounting and audit contexts, this kind of documentation isn't optional; it's essential for quality control and regulatory compliance.

**The Five Critical Utility Functions:**

This cell defines five helper functions that will be used throughout the notebook. The **now_iso()** function captures precise timestamps in international standard format, ensuring you know exactly when each AI interaction occurred. The **sha256_text()** function creates unique digital fingerprints (called cryptographic hashes) of every prompt and response—these fingerprints prove that content hasn't been altered and allow you to trace outputs back to their sources.

The **write_json()** and **append_jsonl()** functions handle saving structured data to files. JSON is a universal format that both humans and computers can read easily. Finally, **get_env_fingerprint()** captures technical details about your computing environment—Python version, operating system, library versions—so someone else could theoretically recreate your exact setup months or years later.

**The Configuration Hash:**

One of the most important innovations here is the configuration hash. The cell takes all your settings (which model you're using, what temperature, what controls are active) and creates a unique identifier. If you run this notebook tomorrow with identical settings, you'll get the same hash. If anything changes—even slightly—the hash changes. This is crucial for reproducibility in professional work.

**The Three Governance Files:**

The cell immediately creates three files in your run directory. The **run_manifest.json** serves as your master record—it contains your run ID, timestamp, all configuration settings, and environment details. The **prompts_log.jsonl** starts as an empty file that will capture every interaction with Claude (we'll add to it later). The **risk_log.json** begins as an empty risk register that will track potential issues as they're identified.

These aren't just bureaucratic requirements—they're your evidence that you exercised appropriate professional skepticism and maintained proper documentation standards.

###4.2.CODE AND IMPLEMENTATION

In [13]:
# Cell 4: Governance — Manifest + Logging Utilities

print("=" * 70)
print("CELL 4: Governance Utilities and Run Manifest")
print("=" * 70)

def now_iso():
    """Return current UTC timestamp in ISO format"""
    from datetime import datetime, timezone
    return datetime.now(timezone.utc).isoformat().replace('+00:00', 'Z')

def sha256_text(text):
    """Compute SHA256 hash of text"""
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

def write_json(filepath, data):
    """Write JSON data to file with pretty formatting"""
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

def append_jsonl(filepath, data):
    """Append JSON object as single line to JSONL file"""
    with open(filepath, 'a', encoding='utf-8') as f:
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

def get_env_fingerprint():
    """Capture environment details for reproducibility"""
    try:
        python_version = platform.python_version()
        system_info = f"{platform.system()} {platform.release()}"

        # Try to get anthropic library version
        try:
            import anthropic
            anthropic_version = anthropic.__version__
        except:
            anthropic_version = "unknown"

        return {
            "python_version": python_version,
            "system": system_info,
            "anthropic_version": anthropic_version,
            "platform": platform.platform()
        }
    except Exception as e:
        return {"error": str(e)}

print("✓ Utility functions defined:")
print("  - now_iso(), sha256_text(), write_json(), append_jsonl(), get_env_fingerprint()")

# Define base configuration
BASE_CONFIG = {
    "chapter": 2,
    "level": 2,
    "level_name": "Reasoners",
    "model": MODEL,
    "temperature": TEMPERATURE,
    "max_tokens": MAX_TOKENS,
    "controls": [
        "strict_json_schema",
        "confidentiality_redaction",
        "not_verified_discipline",
        "facts_vs_assumptions",
        "risk_flagging",
        "level_2_boundary_enforcement"
    ]
}

# Compute config hash
config_str = json.dumps(BASE_CONFIG, sort_keys=True)
config_hash = sha256_text(config_str)
config_hash_prefix = config_hash[:12]

# Generate RUN_ID
RUN_ID = f"{timestamp_str}_{config_hash_prefix}"

print(f"\n✓ Config hash computed: {config_hash_prefix}")
print(f"✓ RUN_ID: {RUN_ID}")

# Create run manifest
env_fingerprint = get_env_fingerprint()
run_manifest = {
    "run_id": RUN_ID,
    "timestamp_utc": now_iso(),
    "chapter": 2,
    "level": 2,
    "model": MODEL,
    "parameters": {
        "temperature": TEMPERATURE,
        "max_tokens": MAX_TOKENS
    },
    "config_hash": config_hash,
    "config": BASE_CONFIG,
    "environment": env_fingerprint,
    "run_directory": str(RUN_DIR)
}

manifest_path = RUN_DIR / "run_manifest.json"
write_json(manifest_path, run_manifest)
print(f"\n✓ run_manifest.json created: {manifest_path}")

# Initialize prompts log (empty JSONL)
prompts_log_path = RUN_DIR / "prompts_log.jsonl"
prompts_log_path.touch()
print(f"✓ prompts_log.jsonl initialized: {prompts_log_path}")

# Initialize risk log (empty register)
risk_log = {
    "run_id": RUN_ID,
    "created_utc": now_iso(),
    "entries": []
}
risk_log_path = RUN_DIR / "risk_log.json"
write_json(risk_log_path, risk_log)
print(f"✓ risk_log.json initialized: {risk_log_path}")

print("\n✓ Governance artifacts ready")
print("Ready to proceed to confidentiality utilities (Cell 5)")

CELL 4: Governance Utilities and Run Manifest
✓ Utility functions defined:
  - now_iso(), sha256_text(), write_json(), append_jsonl(), get_env_fingerprint()

✓ Config hash computed: 22c12306caf0
✓ RUN_ID: 20260111_192247_22c12306caf0

✓ run_manifest.json created: /content/ai_audit_ch2_runs/run_20260111_192247/run_manifest.json
✓ prompts_log.jsonl initialized: /content/ai_audit_ch2_runs/run_20260111_192247/prompts_log.jsonl
✓ risk_log.json initialized: /content/ai_audit_ch2_runs/run_20260111_192247/risk_log.json

✓ Governance artifacts ready
Ready to proceed to confidentiality utilities (Cell 5)


##5.CONFIDENTIALITY  UTILITIES

###5.1.OVERVIEW

**Cell 5: Protecting Confidentiality – Redacting Sensitive Information**

**What This Cell Does:**

This cell creates your first line of defense against accidentally exposing confidential client information. Even though you should only use synthetic or redacted data in this training notebook, mistakes happen. This cell provides automated utilities that scan text for personally identifiable information (PII) and mask it before sending anything to Claude or saving it to your audit trail.

**Understanding the Redaction Function:**

The **redact()** function works like a smart search-and-replace tool. It uses pattern-matching techniques (called regular expressions) to identify common types of sensitive data: email addresses, phone numbers, Social Security numbers, and street addresses. When it finds these patterns, it replaces them with placeholders like [EMAIL_REDACTED] or [PHONE_REDACTED]. The function also attempts to identify potential names by looking for capitalized words appearing together, though this is less reliable and may flag false positives.

**Why Redaction Isn't Perfect:**

Here's a critical point that every professional must understand: automated redaction is helpful but not foolproof. The patterns this cell uses work well for standard formats (like 555-123-4567 for phone numbers or john@example.com for emails), but they can miss unusual formatting or context-specific sensitive information. For example, the function won't catch "John's personal account number is ABC123" because account numbers don't follow predictable patterns. This is why the cell includes prominent warnings reminding you never to paste real client data.

**The Minimum-Necessary Principle:**

The **build_minimum_necessary()** function implements a data minimization strategy borrowed from privacy regulations like GDPR and HIPAA. It takes your input text, runs it through redaction, and then formats it as a clean bullet-point list. It also generates a summary showing what types of information were removed. This serves two purposes: it makes the facts easier for Claude to process in structured format, and it gives you a clear record of what was sanitized.

**The Demonstration:**

The cell includes a live demonstration using synthetic data (a fake client scenario with email, phone, SSN, and address). You'll see the before-and-after comparison, which helps you understand exactly what the redaction function catches and what it might miss. This transparency is intentional—you need to know the limitations of your tools.

###5.2.CODE AND IMPLEMENTATION

In [14]:
# Cell 5: Confidentiality Utilities — Redaction + Minimum-Necessary Facts Builder

print("=" * 70)
print("CELL 5: Confidentiality and Data Minimization Utilities")
print("=" * 70)

def redact(text):
    """
    Redact PII from text. Returns (redacted_text, removed_fields).
    WARNING: This is a heuristic-based redaction; not perfect. Use with caution.
    """
    if not text:
        return text, []

    redacted = text
    removed = []

    # Email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    if re.search(email_pattern, redacted):
        redacted = re.sub(email_pattern, '[EMAIL_REDACTED]', redacted)
        removed.append("email_addresses")

    # Phone numbers (various formats)
    phone_pattern = r'\b(?:\+?1[-.]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b'
    if re.search(phone_pattern, redacted):
        redacted = re.sub(phone_pattern, '[PHONE_REDACTED]', redacted)
        removed.append("phone_numbers")

    # SSN (XXX-XX-XXXX)
    ssn_pattern = r'\b\d{3}-\d{2}-\d{4}\b'
    if re.search(ssn_pattern, redacted):
        redacted = re.sub(ssn_pattern, '[SSN_REDACTED]', redacted)
        removed.append("ssn")

    # Street addresses (simplified heuristic)
    address_pattern = r'\b\d+\s+[A-Z][a-z]+\s+(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct)\.?\b'
    if re.search(address_pattern, redacted):
        redacted = re.sub(address_pattern, '[ADDRESS_REDACTED]', redacted)
        removed.append("street_addresses")

    # Names heuristic (capitalized words >= 2, may have false positives)
    # Only flag, don't auto-redact (too many false positives)
    name_pattern = r'\b[A-Z][a-z]+\s+[A-Z][a-z]+\b'
    if re.search(name_pattern, redacted):
        removed.append("possible_names_detected_not_redacted")

    return redacted, removed

def build_minimum_necessary(text):
    """
    Extract sanitized facts in bullet format, showing what was removed.
    Returns (sanitized_facts_bullets, removed_fields_summary)
    """
    redacted_text, removed = redact(text)

    # Convert to bullet list
    lines = redacted_text.strip().split('\n')
    bullets = []
    for line in lines:
        line = line.strip()
        if line:
            if not line.startswith('•'):
                bullets.append(f"• {line}")
            else:
                bullets.append(line)

    sanitized_facts = '\n'.join(bullets)
    removed_summary = ', '.join(removed) if removed else 'none'

    return sanitized_facts, removed_summary

print("✓ Redaction utilities defined: redact(), build_minimum_necessary()")

# Demo with synthetic data
print("\n" + "="*70)
print("DEMO: Redaction in Action")
print("="*70)

demo_text = """
Client: John Smith at ABC Corp (john.smith@abccorp.com, 555-123-4567)
Address: 123 Main Street, New York
SSN: 123-45-6789
Revenue for Q3 was $5M, up 15% YoY
Identified control deficiency in accounts payable process
"""

print("\nBEFORE REDACTION:")
print(demo_text)

redacted_demo, removed_fields = redact(demo_text)

print("\nAFTER REDACTION:")
print(redacted_demo)

print(f"\nREMOVED FIELDS: {', '.join(removed_fields) if removed_fields else 'none'}")

print("\n⚠ IMPORTANT: Redaction is heuristic-based and may miss PII.")
print("   Always review outputs and use synthetic data when possible.")

print("\nReady to proceed to LLM wrapper (Cell 6)")

CELL 5: Confidentiality and Data Minimization Utilities
✓ Redaction utilities defined: redact(), build_minimum_necessary()

DEMO: Redaction in Action

BEFORE REDACTION:

Client: John Smith at ABC Corp (john.smith@abccorp.com, 555-123-4567)
Address: 123 Main Street, New York
SSN: 123-45-6789
Revenue for Q3 was $5M, up 15% YoY
Identified control deficiency in accounts payable process


AFTER REDACTION:

Client: John Smith at ABC Corp ([EMAIL_REDACTED], [PHONE_REDACTED])
Address: [ADDRESS_REDACTED], New York
SSN: [SSN_REDACTED]
Revenue for Q3 was $5M, up 15% YoY
Identified control deficiency in accounts payable process


REMOVED FIELDS: email_addresses, phone_numbers, ssn, street_addresses, possible_names_detected_not_redacted

⚠ IMPORTANT: Redaction is heuristic-based and may miss PII.
   Always review outputs and use synthetic data when possible.

Ready to proceed to LLM wrapper (Cell 6)


##6.CLAUDE WRAPPER

###6.1.OVERVIEW

**Cell 6: The AI Wrapper – Enforcing Disciplined Reasoning**

**What This Cell Does:**

This is arguably the most important cell in the entire notebook. It creates the **call_reasoner()** function, which serves as your controlled interface to Claude. Rather than sending raw questions to the AI and hoping for useful answers, this wrapper enforces strict rules about how Claude must respond, what format it must use, and what safety checks must pass before you see any output.

**The System Prompt – Your Instruction Manual for Claude:**

Think of the system prompt as a detailed job description you're giving to an employee. It explicitly tells Claude what Level 2 reasoning means: structure thinking, identify gaps, generate hypotheses, but never claim to have performed procedures or verified evidence. The prompt enforces a mandatory JSON output format with eight required fields that must appear in exact order. This isn't arbitrary—structured outputs are easier to audit, validate, and integrate into your workflow than free-form text.

**Facts, Assumptions, and Open Questions:**

The system prompt makes a critical distinction that many AI users overlook. The **facts_provided** field must contain only what you explicitly stated in your input. The **assumptions** field captures what Claude inferred or filled in mentally—these must be validated by you, the professional. The **open_questions** field is required because no scenario ever has complete information. By forcing Claude to articulate what's missing, you're building professional skepticism directly into the AI's output.

**Automated Risk Flagging:**

After Claude responds, the wrapper doesn't just accept the output blindly. It runs three automated quality checks. First, if Claude forgot to include open questions, the system flags a "missing_facts" risk—suggesting the analysis might be overconfident. Second, if the response mentions authorities like ASC, PCAOB, or SEC without marking them "Not verified," it flags a high-severity hallucination risk. Third, if the analysis uses conclusive language like "therefore" or "must be," it flags an overreach risk, reminding you this is reasoning support, not a final conclusion.

**The Audit Trail Connection:**

Every call to this function automatically logs a redacted version of the prompt and response to your prompts_log.jsonl file, along with cryptographic hashes. It also updates the risk_log.json with any flagged issues. This happens transparently in the background, ensuring your audit trail stays complete without extra effort.

###6.2.CODE AND IMPLEMENTATION

In [19]:
# Cell 6: LLM Wrapper — Level 2 Reasoner Call with Strict JSON + Risk Flags

print("=" * 70)
print("CELL 6: LLM Wrapper for Level 2 Reasoned Structuring")
print("=" * 70)

# Define strict JSON schema enforcement
SYSTEM_PROMPT = """You are a Level 2 Reasoner providing structured reasoning support for US CPAs and auditors.

LEVEL 2 BOUNDARY (STRICT):
- You provide structured reasoning support ONLY
- You do NOT perform procedures, verify evidence, or provide final conclusions
- You do NOT act as an autonomous agent or orchestrator
- You structure thinking: hypotheses, issue lists, research scaffolds, disconfirming-evidence questions

OUTPUT FORMAT (MANDATORY):
Return ONLY valid JSON with these keys in EXACT order. No markdown, no code blocks, no explanation - JUST the JSON object:

{
  "task": "brief task description",
  "facts_provided": ["fact 1", "fact 2"],
  "assumptions": ["assumption 1", "assumption 2"],
  "open_questions": ["question 1", "question 2"],
  "analysis": "reasoning structure notes (WHY these hypotheses/questions, NOT technical conclusions)",
  "risks": [
    {"type": "confidentiality|independence|hallucination|missing_facts|qc|prompt_injection|overreach|other",
     "severity": "low|medium|high",
     "note": "explanation"}
  ],
  "draft_output": "the structured deliverable with disclaimer",
  "verification_status": "Not verified",
  "questions_to_verify": ["verification question 1"]
}

CRITICAL RULES:
1. facts_provided = only what was explicitly stated in input
2. assumptions = what you inferred (must be validated by user)
3. open_questions = REQUIRED; what's missing
4. analysis = reasoning notes, NOT conclusions
5. draft_output MUST begin with: "NOT ACCOUNTING/AUDIT/TAX ADVICE. CPA review and engagement sign-off required."
6. If you reference ASC/PCAOB/AICPA/SEC, keep "Not verified" and populate questions_to_verify
7. Never claim procedures were performed or evidence exists unless stated in facts_provided
8. Be skeptical: always include disconfirming-evidence questions

Return ONLY the raw JSON object. Do not wrap it in markdown code blocks. Do not add any text before or after the JSON."""

def clean_json_response(text):
    """
    Clean potential markdown formatting from JSON response
    """
    # Remove markdown code blocks if present
    text = text.strip()

    # Remove ```json and ``` markers
    if text.startswith('```json'):
        text = text[7:]
    elif text.startswith('```'):
        text = text[3:]

    if text.endswith('```'):
        text = text[:-3]

    # Strip whitespace again
    text = text.strip()

    return text

def call_reasoner(task_name, user_prompt, facts_bullets):
    """
    Call Claude with Level 2 system prompt, enforce strict JSON, add risk flags.
    Returns parsed JSON or None on failure.
    """
    print(f"\n{'─'*70}")
    print(f"Calling reasoner: {task_name}")
    print(f"{'─'*70}")

    # Construct user message
    full_prompt = f"""TASK: {task_name}

FACTS PROVIDED (minimum necessary):
{facts_bullets}

{user_prompt}

CRITICAL: Return ONLY the raw JSON object as specified in the system prompt. No markdown code blocks, no explanations, no preamble. Start your response with {{ and end with }}."""

    # Redact prompt for logging
    prompt_redacted, _ = redact(full_prompt)
    prompt_hash = sha256_text(full_prompt)

    try:
        # Call Anthropic API
        response = client.messages.create(
            model=MODEL,
            max_tokens=MAX_TOKENS,
            temperature=TEMPERATURE,
            system=SYSTEM_PROMPT,
            messages=[{"role": "user", "content": full_prompt}]
        )

        # Check if response was truncated
        if response.stop_reason == "max_tokens":
            print(f"⚠ WARNING: Response hit max_tokens limit ({MAX_TOKENS})")
            print(f"⚠ Output may be incomplete. Consider increasing MAX_TOKENS in Cell 3.")
            return None

        response_text = response.content[0].text.strip()

        # Clean any markdown formatting
        cleaned_response = clean_json_response(response_text)

        # Redact response for logging
        response_redacted, _ = redact(cleaned_response)
        response_hash = sha256_text(cleaned_response)

        # Log to prompts_log.jsonl
        log_entry = {
            "timestamp_utc": now_iso(),
            "task_name": task_name,
            "prompt_redacted": prompt_redacted[:500] + "..." if len(prompt_redacted) > 500 else prompt_redacted,
            "prompt_hash": prompt_hash,
            "response_redacted": response_redacted[:500] + "..." if len(response_redacted) > 500 else response_redacted,
            "response_hash": response_hash,
            "model": MODEL,
            "temperature": TEMPERATURE
        }
        append_jsonl(prompts_log_path, log_entry)

        # Parse JSON
        try:
            result = json.loads(cleaned_response)
        except json.JSONDecodeError as e:
            print(f"⚠ JSON parse failed: {str(e)}")
            print(f"⚠ First 200 chars of response: {cleaned_response[:200]}")

            # Retry once with explicit fix request
            print("⚠ Attempting fix with explicit instruction...")
            fix_prompt = f"""The following response should be valid JSON but has formatting issues.
Return ONLY the corrected JSON object with no markdown code blocks, no explanation, no preamble.
Start with {{ and end with }}.

Response to fix:
{cleaned_response}"""

            fix_response = client.messages.create(
                model=MODEL,
                max_tokens=MAX_TOKENS,
                temperature=0,
                messages=[{"role": "user", "content": fix_prompt}]
            )

            fixed_text = fix_response.content[0].text.strip()
            fixed_text = clean_json_response(fixed_text)

            result = json.loads(fixed_text)
            print("✓ JSON fixed and parsed")

        # Automated risk flags
        automated_risks = []

        # Risk: missing open_questions
        if not result.get("open_questions") or len(result.get("open_questions", [])) == 0:
            automated_risks.append({
                "type": "missing_facts",
                "severity": "medium",
                "note": "No open_questions provided; facts may be incomplete"
            })

        # Risk: authority-like tokens without verification
        authority_tokens = ["ASC", "PCAOB", "AICPA", "SEC", "GAAS", "FASB"]
        response_upper = cleaned_response.upper()
        found_authorities = [tok for tok in authority_tokens if tok in response_upper]

        if found_authorities and result.get("verification_status") != "Not verified":
            automated_risks.append({
                "type": "hallucination",
                "severity": "high",
                "note": f"Authority-like tokens detected ({', '.join(found_authorities)}) but verification_status not set to 'Not verified'"
            })

        if found_authorities and (not result.get("questions_to_verify") or len(result.get("questions_to_verify", [])) == 0):
            automated_risks.append({
                "type": "hallucination",
                "severity": "high",
                "note": f"Authority-like tokens detected ({', '.join(found_authorities)}) but questions_to_verify is empty"
            })

        # Risk: analysis sounds like conclusion (heuristic)
        analysis_text = result.get("analysis", "").lower()
        conclusion_words = ["therefore", "conclude", "must be", "is certain", "definitively", "proven"]
        if any(word in analysis_text for word in conclusion_words):
            automated_risks.append({
                "type": "overreach",
                "severity": "medium",
                "note": "Analysis contains conclusion-like language; ensure it's framed as reasoning structure only"
            })

        # Merge automated risks with model-provided risks
        existing_risks = result.get("risks", [])
        all_risks = existing_risks + automated_risks
        result["risks"] = all_risks

        # Update risk_log.json
        risk_log_data = json.loads(risk_log_path.read_text())
        risk_log_data["entries"].append({
            "timestamp_utc": now_iso(),
            "task_name": task_name,
            "prompt_hash": prompt_hash,
            "response_hash": response_hash,
            "risks": all_risks
        })
        write_json(risk_log_path, risk_log_data)

        print(f"✓ Reasoner call completed")
        print(f"  - Facts provided: {len(result.get('facts_provided', []))}")
        print(f"  - Assumptions: {len(result.get('assumptions', []))}")
        print(f"  - Open questions: {len(result.get('open_questions', []))}")
        print(f"  - Risks flagged: {len(all_risks)}")

        return result

    except Exception as e:
        print(f"✗ Error calling reasoner: {e}")
        import traceback
        print(f"✗ Traceback: {traceback.format_exc()}")
        return None

print("✓ Reasoner wrapper defined: call_reasoner()")
print("✓ JSON cleaning utility: clean_json_response()")

# Smoke test
print("\n" + "="*70)
print("SMOKE TEST: Reasoner Call")
print("="*70)

test_result = call_reasoner(
    task_name="Smoke Test",
    user_prompt="Generate a simple example showing the JSON structure with at least 2 facts, 1 assumption, and 2 open questions.",
    facts_bullets="• Test scenario for smoke test\n• No real data used"
)

if test_result:
    print("\n✓ SMOKE TEST PASSED")
    print(f"  Task: {test_result.get('task')}")
    print(f"  Verification status: {test_result.get('verification_status')}")
else:
    print("\n✗ SMOKE TEST FAILED - Check API key and model availability")

print("\nReady to proceed to mini-case builders (Cell 7)")

CELL 6: LLM Wrapper for Level 2 Reasoned Structuring
✓ Reasoner wrapper defined: call_reasoner()
✓ JSON cleaning utility: clean_json_response()

SMOKE TEST: Reasoner Call

──────────────────────────────────────────────────────────────────────
Calling reasoner: Smoke Test
──────────────────────────────────────────────────────────────────────
✓ Reasoner call completed
  - Facts provided: 2
  - Assumptions: 1
  - Open questions: 2
  - Risks flagged: 2

✓ SMOKE TEST PASSED
  Task: Smoke test to validate JSON structure and Level 2 reasoning boundaries
  Verification status: Not verified

Ready to proceed to mini-case builders (Cell 7)


##7.MINI CASE BUILDER

###7.1.OVERVIEW

**Cell 7: Building the Mini-Cases – Real-World Scenarios for Practice**

**What This Cell Does:**

This cell defines four realistic professional scenarios that demonstrate how Level 2 AI reasoning applies to different areas of accounting and audit practice. Rather than abstract examples, these mini-cases mirror situations you'll encounter in actual engagements: unusual financial statement variances, internal control weaknesses, complex tax positions, and training challenges. Each scenario is intentionally incomplete—missing key facts that would be needed for final conclusions—to demonstrate how AI helps structure your thinking when information is imperfect.

**Mini-Case 1: Financial Statement Audit (Revenue Variance):**

You're reviewing a manufacturing client where Q4 revenue jumped 18 percent but gross margin dropped 6 percentage points. Management blames it on a new product line with lower margins, but you don't have detailed sales mix data yet. This case teaches you how to generate competing hypotheses (maybe it's the product mix, maybe it's pricing pressure, maybe there's a revenue recognition issue) and how to develop disconfirming-evidence questions that test each hypothesis. The key lesson: never accept the first explanation without considering alternatives.

**Mini-Case 2: SOX/ICFR (IT-Dependent Manual Control):**

A monthly inventory reconciliation control has multiple red flags: the person doing the reconciliation also has access to warehouse transactions (segregation of duties issue), one reconciliation was completed late, another showed an unexplained difference marked "timing" with no follow-up. This scenario demonstrates issue-spotting—teaching you how to identify both design deficiencies (problems with how the control is set up) and operating effectiveness deficiencies (problems with how it's actually being performed).

**Mini-Case 3: Tax/ASC 740 (Uncertain Tax Position):**

A software company has an employee working remotely in State X, generating two million dollars in revenue from that state's customers, but management hasn't filed a state tax return because they claim no nexus exists. This case shows how to scaffold research questions without fabricating citations, how to identify what evidence you'd need from the client and tax advisor, and how to maintain "Not verified" discipline when discussing technical authorities.

**Mini-Case 4: Teaching and Methodology:**

Your firm wants to train 50 staff members on using AI responsibly. This meta-case demonstrates how to create training rubrics, design quizzes that test understanding of key concepts (facts versus assumptions, verification requirements), and identify risks in AI adoption like overconfidence or lack of skepticism.

###7.2.CODE AND IMPLEMENTATION

In [20]:
# Cell 7: Mini-Case Builders (4 Scenarios Aligned to Level 2)

print("=" * 70)
print("CELL 7: Mini-Case Scenario Definitions")
print("=" * 70)

def minicase_fs_audit_variance():
    """
    Mini-Case 1: FS Audit — Revenue variance hypotheses + disconfirming evidence
    """
    task_name = "FS_Audit_Revenue_Variance_Hypotheses"

    facts = """• Client: Manufacturing company, calendar year-end 12/31/2024
- Q4 2024 revenue: $45M (PY Q4: $38M, +18.4% YoY)
- Gross margin Q4 2024: 22% (PY Q4: 28%, -600 bps)
- New product line launched Q3 2024 with lower margins
- Sales mix data: not yet available for Q4
- Management explanation: "Volume growth drove revenue increase; new product has lower margin profile"
- No unusual journal entries identified in preliminary analytics
- Accounts receivable days outstanding: 48 (PY: 42)
- Customer concentration: top 3 customers = 35% of revenue (PY: 28%)"""

    user_prompt = """Your task:
1. Generate variance hypotheses for BOTH revenue increase and gross margin decrease
2. For each hypothesis, list disconfirming-evidence questions (professional skepticism)
3. Draft a follow-up plan skeleton showing:
   - What procedures to consider (do NOT claim they were performed)
   - What evidence to request
   - What analytics to run
4. Flag any independence, confidentiality, or hallucination risks
5. Identify open questions that need management/auditor follow-up

IMPORTANT: This is reasoning support only. Do not conclude the variance is or isn't a risk."""

    return task_name, facts, user_prompt


def minicase_sox_icfr_issue_spotting():
    """
    Mini-Case 2: SOX/ICFR — Issue-spotting for IT-dependent manual control
    """
    task_name = "SOX_ICFR_Issue_Spotting_IT_Manual_Control"

    facts = """• Control: Monthly reconciliation of inventory subledger to GL
- Control owner: Inventory Accountant (not segregated from warehouse transactions)
- Frequency: Monthly, by 5th business day
- IT system: Legacy ERP system with manual interfaces
- Control description: "Compare inventory subledger report to GL balance; investigate differences >$10K"
- Evidence: Reconciliation spreadsheet with sign-off
- Testing period: July-December 2024
- Observation: November reconciliation completed on 8th business day (late)
- Observation: October reconciliation showed $12K difference marked "timing" with no follow-up documentation
- IT general controls: Annual review, no deficiencies noted in 2023
- Automated controls: None for this process"""

    user_prompt = """Your task:
1. Issue-spot potential design and operating effectiveness deficiencies
2. For each issue, identify missing control objective clarity or design elements
3. Propose a test approach (draft only; do not claim tests were performed)
4. Flag any QC (quality control), missing facts, or overreach risks
5. Identify open questions for the internal audit team

IMPORTANT: This is issue-spotting support only. Do not conclude the control is or isn't effective."""

    return task_name, facts, user_prompt


def minicase_tax_asc740_scaffold():
    """
    Mini-Case 3: Tax/ASC 740 — Research scaffold without citations
    """
    task_name = "Tax_ASC740_Research_Scaffold_Uncertain_Position"

    facts = """• Entity: US C-corp, software industry
- Issue: Tax treatment of R&D capitalization under updated IRS regulations (2024)
- Prior treatment: Expensed R&D costs immediately
- New requirement: Capitalize and amortize R&D over 5/15 years (domestic/foreign)
- Impact: Increase in taxable income; potential deferred tax asset
- Question: Recognition threshold for uncertain tax position related to nexus in State X
- State X nexus: Employee working remotely; no physical office; $2M revenue from State X customers
- Management position: No nexus; no state tax return filed
- Tax advisor: "More likely than not" threshold unclear for remote-employee nexus post-Wayfair
- No specific facts pattern on prior case law or IRS/state guidance"""

    user_prompt = """Your task:
1. Frame the ASC 740 research questions (do NOT provide ASC citations; use "Not verified" placeholders)
2. Identify evidence needs (what documents/data to request from client and tax advisor)
3. Draft a memo shell structure with "Not verified" warnings for any authority-like statements
4. Create a provision binder request list (what should be in the tax provision workpapers)
5. Flag hallucination risks if any authority-like language is used
6. Populate questions_to_verify with specific items to check in ASC/IRS guidance

IMPORTANT: This is a research scaffold only. Do not conclude the position is or isn't supportable."""

    return task_name, facts, user_prompt


def minicase_teaching_methodology():
    """
    Mini-Case 4: Teaching — Level 2 training rubric + quiz
    """
    task_name = "Teaching_Level2_Training_Rubric_Quiz"

    facts = """• Training context: CPA firm, 50-person firm, mix of seniors and managers
- Goal: Train staff on Level 2 AI reasoners (structured reasoning support)
- Current state: Staff have used ChatGPT for research; no formal AI training
- Concerns: Over-reliance on AI; lack of professional skepticism; not verifying AI output
- Desired outcome: Staff understand facts vs assumptions, "Not verified" discipline, open questions requirement
- Training format: 2-hour workshop + take-home exercises"""

    user_prompt = """Your task:
1. Create a training rubric for evaluating Level 2 AI outputs, including:
   - Facts vs assumptions policing checklist
   - "Not verified" discipline for authority-like statements
   - Open questions requirement (skepticism)
   - QC (quality control) sign-off process
2. Draft a short quiz (5 questions) testing these concepts
   - Include 2 scenario-based questions with "good" vs "bad" AI output examples
   - Quiz should be validated by instructor later (flag as draft)
3. Identify risks in AI training (e.g., over-confidence, prompt injection, confidentiality)
4. List open questions for workshop design

IMPORTANT: This is training material draft only. Instructor review required before use."""

    return task_name, facts, user_prompt


# Register all mini-cases
MINI_CASES = [
    minicase_fs_audit_variance,
    minicase_sox_icfr_issue_spotting,
    minicase_tax_asc740_scaffold,
    minicase_teaching_methodology
]

print(f"✓ {len(MINI_CASES)} mini-case builders loaded:")
for i, case_func in enumerate(MINI_CASES, 1):
    task_name, _, _ = case_func()
    print(f"  {i}. {task_name}")

print("\nReady to proceed to running mini-cases (Cell 8)")

CELL 7: Mini-Case Scenario Definitions
✓ 4 mini-case builders loaded:
  1. FS_Audit_Revenue_Variance_Hypotheses
  2. SOX_ICFR_Issue_Spotting_IT_Manual_Control
  3. Tax_ASC740_Research_Scaffold_Uncertain_Position
  4. Teaching_Level2_Training_Rubric_Quiz

Ready to proceed to running mini-cases (Cell 8)


##8.EXECUTION

###8.1.OVERVIEW

**Cell 8: Running the Demonstrations – Seeing Level 2 Reasoning in Action**

**What This Cell Does:**

This is where theory becomes practice. Cell 8 takes the four mini-cases you just learned about and actually runs them through Claude, generating complete outputs for each scenario. You'll see the AI in action, structuring reasoning, flagging risks, and producing draft deliverables—all while the governance systems capture everything in your audit trail. This cell demonstrates the full workflow from raw facts to documented output, showing you what professional-grade AI assistance looks like.

**The Processing Loop:**

The cell processes each mini-case sequentially, and you can watch the progress in real-time. For each scenario, it first applies the redaction utilities from Cell 5 to ensure any sensitive information is masked. Then it calls the reasoner wrapper from Cell 6, which sends the scenario to Claude with strict instructions. The AI responds with structured JSON containing facts, assumptions, open questions, analysis, risks, and a draft deliverable. This entire interaction—prompt, response, and metadata—gets logged to your prompts_log.jsonl file with cryptographic hashes.

**Two Output Formats for Different Purposes:**

Each mini-case generates two files in your deliverables folder. The JSON file preserves the raw structured data—perfect for programmatic analysis or quality control software. The TXT file transforms that data into a human-readable report with clear sections, disclaimer language, and formatting that makes it easy for a reviewing CPA to evaluate. Both files serve the same content but optimize for different audiences: machines versus humans.

**The Results Summary Table:**

After processing all four cases, the cell displays a plain-text table summarizing key quality metrics. You'll see how many open questions each case generated (more is better—it shows professional skepticism), what the highest risk severity was (high-severity risks demand immediate attention), and whether any overreach was detected (cases where Claude might have stepped beyond reasoning support into conclusion territory). This table gives supervisors a quick way to triage which outputs need the most careful review.

**The Minimum Standard Document:**

Finally, the cell creates a reference document called level2_minimum_standard.txt that codifies the seven quality requirements every Level 2 output must meet. This becomes your checklist for evaluating not just these four demos but any future AI-assisted work you do.

###8.2.CODE AND IMPLEMENTATION

In [21]:
# Cell 8: Run 4 Mini-Case Demos + Save Deliverables

print("=" * 70)
print("CELL 8: Running Mini-Case Demonstrations")
print("=" * 70)

results_summary = []

for i, case_func in enumerate(MINI_CASES, 1):
    print(f"\n{'='*70}")
    print(f"MINI-CASE {i}/{len(MINI_CASES)}")
    print(f"{'='*70}")

    task_name, facts_text, user_prompt = case_func()

    # Build minimum-necessary facts (redacted)
    facts_bullets, removed_summary = build_minimum_necessary(facts_text)
    print(f"Facts prepared (removed: {removed_summary})")

    # Call reasoner
    result = call_reasoner(task_name, user_prompt, facts_bullets)

    if result:
        # Save JSON deliverable
        json_path = DELIVERABLES_DIR / f"{task_name}_output.json"
        write_json(json_path, result)
        print(f"\n✓ JSON saved: {json_path}")

        # Create human-readable draft
        draft_lines = [
            "="*70,
            f"TASK: {task_name}",
            "="*70,
            "",
            "🚨 NOT ACCOUNTING/AUDIT/TAX ADVICE 🚨",
            "CPA review and engagement sign-off required.",
            "",
            "FACTS PROVIDED:",
            *[f"  {fact}" for fact in result.get('facts_provided', [])],
            "",
            "ASSUMPTIONS (MUST BE VALIDATED):",
            *[f"  {assumption}" for assumption in result.get('assumptions', [])],
            "",
            "OPEN QUESTIONS:",
            *[f"  {q}" for q in result.get('open_questions', [])],
            "",
            "ANALYSIS (reasoning structure notes):",
            textwrap.fill(result.get('analysis', ''), width=70),
            "",
            "RISKS FLAGGED:",
            *[f"  - [{r['severity'].upper()}] {r['type']}: {r['note']}" for r in result.get('risks', [])],
            "",
            f"VERIFICATION STATUS: {result.get('verification_status')}",
            "",
            "QUESTIONS TO VERIFY:",
            *[f"  {q}" for q in result.get('questions_to_verify', [])],
            "",
            "-"*70,
            "DRAFT OUTPUT:",
            "-"*70,
            result.get('draft_output', ''),
            ""
        ]

        draft_text = '\n'.join(draft_lines)
        draft_path = DELIVERABLES_DIR / f"{task_name}_draft.txt"
        draft_path.write_text(draft_text, encoding='utf-8')
        print(f"✓ Draft saved: {draft_path}")

        # Collect summary
        risks = result.get('risks', [])
        highest_risk = 'none'
        if risks:
            severity_order = {'high': 3, 'medium': 2, 'low': 1}
            highest_risk = max(risks, key=lambda r: severity_order.get(r['severity'], 0))['severity']

        overreach_flagged = any(r['type'] == 'overreach' for r in risks)

        results_summary.append({
            'case': task_name,
            'open_questions': len(result.get('open_questions', [])),
            'highest_risk': highest_risk,
            'overreach_flagged': 'Yes' if overreach_flagged else 'No'
        })
    else:
        print(f"\n✗ Failed to process {task_name}")
        results_summary.append({
            'case': task_name,
            'open_questions': 0,
            'highest_risk': 'ERROR',
            'overreach_flagged': 'N/A'
        })

# Create Level 2 minimum standard document
standard_text = """Level 2 Reasoners: Minimum Quality Standard
==============================================

Level 2 AI reasoners provide structured reasoning support for accounting and audit professionals.
To ensure safe and effective use, every Level 2 output must meet these minimum standards:

1. FACTS VS ASSUMPTIONS DISCIPLINE
   ✓ Explicitly separate what was stated (facts_provided) from what was inferred (assumptions)
   ✓ All assumptions must be validated by human professionals
   ✓ Never claim procedures were performed unless stated in facts

2. "NOT VERIFIED" GATE
   ✓ Any reference to ASC, PCAOB, AICPA, SEC, GAAS, or other authorities must be flagged "Not verified"
   ✓ Populate questions_to_verify with specific items to check in authoritative literature
   ✓ Never fabricate citations or requirements

3. PROFESSIONAL SKEPTICISM (DISCONFIRMING EVIDENCE)
   ✓ Every hypothesis must include disconfirming-evidence questions
   ✓ Open questions are REQUIRED to identify missing facts
   ✓ Challenge assumptions and look for alternative explanations

4. LEVEL 2 BOUNDARY ENFORCEMENT
   ✓ Reasoning support ONLY; no autonomous execution
   ✓ No final conclusions or opinions
   ✓ No procedures performed or evidence verified
   ✓ Explicitly state: "NOT ADVICE; CPA review required"

5. RISK FLAGGING
   ✓ Identify confidentiality, independence, hallucination, missing facts, QC, overreach risks
   ✓ Document risks in structured format with severity levels
   ✓ Update risk log for auditability

6. HUMAN REVIEW AND SIGN-OFF
   ✓ All outputs are drafts requiring CPA review
   ✓ Engagement sign-off required before use in real engagements
   ✓ Maintain audit trail (run manifest, prompts log, risk log)

7. REPRODUCIBILITY
   ✓ Config hash + environment fingerprint for every run
   ✓ Cryptographic hashes for prompts and responses
   ✓ Zip bundle with complete audit trail

Remember: Capability ↑ ⇒ Risk ↑ ⇒ Controls ↑

This is not a substitute for professional judgment. AI is a tool to structure thinking, not replace thinking.
"""

standard_path = DELIVERABLES_DIR / "level2_minimum_standard.txt"
standard_path.write_text(standard_text, encoding='utf-8')
print(f"\n✓ Level 2 minimum standard saved: {standard_path}")

# Print summary table
print("\n" + "="*70)
print("MINI-CASE RESULTS SUMMARY")
print("="*70)
print(f"{'Case':<45} {'Open Q':<8} {'Risk':<8} {'Overreach'}")
print("-"*70)
for row in results_summary:
    print(f"{row['case']:<45} {row['open_questions']:<8} {row['highest_risk']:<8} {row['overreach_flagged']}")
print("="*70)

print("\nReady to proceed to user exercise (Cell 9)")

CELL 8: Running Mini-Case Demonstrations

MINI-CASE 1/4
Facts prepared (removed: none)

──────────────────────────────────────────────────────────────────────
Calling reasoner: FS_Audit_Revenue_Variance_Hypotheses
──────────────────────────────────────────────────────────────────────
✓ Reasoner call completed
  - Facts provided: 9
  - Assumptions: 8
  - Open questions: 15
  - Risks flagged: 5

✓ JSON saved: /content/ai_audit_ch2_runs/run_20260111_192247/deliverables/FS_Audit_Revenue_Variance_Hypotheses_output.json
✓ Draft saved: /content/ai_audit_ch2_runs/run_20260111_192247/deliverables/FS_Audit_Revenue_Variance_Hypotheses_draft.txt

MINI-CASE 2/4
Facts prepared (removed: possible_names_detected_not_redacted)

──────────────────────────────────────────────────────────────────────
Calling reasoner: SOX_ICFR_Issue_Spotting_IT_Manual_Control
──────────────────────────────────────────────────────────────────────
✓ Reasoner call completed
  - Facts provided: 11
  - Assumptions: 8
  - Open 

##9.USER EXERCISE

###9.1.OVERVIEW

**Cell 9: Your Turn – Applying AI Reasoning to Your Own Scenario**

**What This Cell Does:**

After observing four demonstration cases, now you get hands-on experience. Cell 9 transforms the notebook from a passive learning tool into an interactive workspace where you can input your own scenario and receive structured reasoning support. This is where the training becomes practical—you'll see how the same governance controls, redaction utilities, and quality checks apply to scenarios you create, not just pre-built examples.

**The Three-Step Safe Intake Process:**

The cell guides you through a carefully designed input process. Step one asks you to type or paste your scenario, with prominent reminders to use only synthetic or heavily redacted data. The interface accepts multi-line input, so you can provide as much context as needed. Step two automatically runs the redaction function and shows you what was removed—emails, phone numbers, addresses, or other PII. You'll see both the cleaned version and a summary of removed fields. Step three asks for explicit confirmation before proceeding, giving you a chance to review and cancel if anything looks problematic.

**Choosing Your Exercise Type:**

Not all professional scenarios require the same kind of reasoning support. The cell offers three distinct exercise types aligned to common CPA workflows. **Variance hypotheses** helps you analyze unexpected changes in financial statement line items—perfect for audit analytics or monthly financial reviews. **Issue spotting** guides you through identifying control deficiencies in SOX compliance or internal audit contexts. **ASC scaffold** structures your approach to complex accounting research questions, especially useful for technical accounting memos or tax provision work.

**Real-Time Output Generation:**

Once you select your exercise type, the cell calls the same reasoner wrapper used in the demos, applying identical quality controls and documentation standards. Within seconds, you'll receive structured output showing what facts Claude extracted from your scenario, what assumptions it made that you need to validate, what questions remain unanswered, and what risks exist in the analysis. This isn't a generic response—it's tailored specifically to your input and your chosen exercise type.

**Your Personal Deliverables:**

The cell saves two files with your results: user_exercise_output.json for the structured data and user_exercise_output.txt for the readable report. These files join the demo outputs in your deliverables folder, creating a complete portfolio of both training examples and your own work.

###9.2.CODE AND IMPLEMENTATION

In [ ]:
# Cell 9: User Exercise — Your Turn (Safe Intake + Reasoned Scaffold)

print("=" * 70)
print("CELL 9: User Exercise — Your Own Scenario")
print("=" * 70)

print("""
This cell lets you input your own scenario and get Level 2 reasoning support.

⚠ CRITICAL REMINDERS:
1. Use REDACTED or SYNTHETIC data only (no real client information)
2. Even with redaction, be cautious about what you input
3. All outputs are DRAFTS requiring CPA review
""")

# Get user scenario
print("\nStep 1: Enter your scenario (use synthetic/redacted data)")
print("Press Enter twice when done:\n")

user_lines = []
while True:
    line = input()
    if line == "" and len(user_lines) > 0 and user_lines[-1] == "":
        break
    user_lines.append(line)

user_scenario = '\n'.join(user_lines).strip()

if not user_scenario:
    print("\n⚠ No scenario entered. Skipping user exercise.")
else:
    # Redact the scenario
    print("\nStep 2: Applying redaction...")
    redacted_scenario, removed_fields = redact(user_scenario)

    print("\nREDACTION SUMMARY:")
    print(f"  Removed fields: {', '.join(removed_fields) if removed_fields else 'none'}")

    if removed_fields:
        print("\n⚠ PII detected and redacted. Review output carefully.")
        print("\nREDACTED SCENARIO:")
        print(redacted_scenario)
        confirm = input("\nContinue with this redacted scenario? (yes/no): ").strip().lower()
        if confirm != 'yes':
            print("User exercise cancelled.")
            redacted_scenario = None

    if redacted_scenario:
        # Build sanitized facts
        facts_bullets, _ = build_minimum_necessary(redacted_scenario)

        # Get exercise type
        print("\nStep 3: Select exercise type")
        print("  1. variance_hypotheses (revenue/expense variance analysis)")
        print("  2. issue_spotting (internal control deficiencies)")
        print("  3. asc_scaffold (tax/accounting research framework)")

        exercise_choice = input("\nEnter 1, 2, or 3: ").strip()

        exercise_map = {
            '1': ('variance_hypotheses', """Generate variance hypotheses and disconfirming-evidence questions for the scenario.
Include: hypotheses list, skepticism questions, follow-up plan skeleton, risk flags."""),
            '2': ('issue_spotting', """Identify potential control design or operating effectiveness issues.
Include: issue list, missing control elements, proposed test approach (draft), risk flags."""),
            '3': ('asc_scaffold', """Create an accounting research scaffold with "Not verified" placeholders.
Include: research questions, evidence needs, memo shell, questions_to_verify.""")
        }

        if exercise_choice in exercise_map:
            exercise_type, exercise_prompt = exercise_map[exercise_choice]
            task_name = f"User_Exercise_{exercise_type}"

            print(f"\nRunning exercise: {exercise_type}")

            # Call reasoner
            result = call_reasoner(task_name, exercise_prompt, facts_bullets)

            if result:
                # Save JSON
                json_path = DELIVERABLES_DIR / "user_exercise_output.json"
                write_json(json_path, result)
                print(f"\n✓ JSON saved: {json_path}")

                # Save readable output
                output_lines = [
                    "="*70,
                    "USER EXERCISE OUTPUT",
                    "="*70,
                    "",
                    "🚨 NOT ACCOUNTING/AUDIT/TAX ADVICE 🚨",
                    "This is a DRAFT requiring CPA review.",
                    "",
                    f"Exercise type: {exercise_type}",
                    "",
                    "YOUR SANITIZED FACTS:",
                    facts_bullets,
                    "",
                    "ASSUMPTIONS (MUST BE VALIDATED):",
                    *[f"  {a}" for a in result.get('assumptions', [])],
                    "",
                    "OPEN QUESTIONS:",
                    *[f"  {q}" for q in result.get('open_questions', [])],
                    "",
                    "ANALYSIS:",
                    textwrap.fill(result.get('analysis', ''), width=70),
                    "",
                    "RISKS:",
                    *[f"  - [{r['severity'].upper()}] {r['type']}: {r['note']}" for r in result.get('risks', [])],
                    "",
                    f"VERIFICATION STATUS: {result.get('verification_status')}",
                    "",
                    "QUESTIONS TO VERIFY:",
                    *[f"  {q}" for q in result.get('questions_to_verify', [])],
                    "",
                    "-"*70,
                    "DRAFT OUTPUT:",
                    "-"*70,
                    result.get('draft_output', ''),
                    ""
                ]

                output_text = '\n'.join(output_lines)
                txt_path = DELIVERABLES_DIR / "user_exercise_output.txt"
                txt_path.write_text(output_text, encoding='utf-8')
                print(f"✓ Readable output saved: {txt_path}")

                print("\n" + "="*70)
                print("USER EXERCISE COMPLETE")
                print("="*70)
                print(f"Redaction applied: {len(removed_fields) > 0}")
                print(f"Open questions: {len(result.get('open_questions', []))}")
                print(f"Risks flagged: {len(result.get('risks', []))}")
                print("\n⚠ Remember: Review output carefully and obtain CPA sign-off before use.")
            else:
                print("\n✗ Exercise failed")
        else:
            print("\n⚠ Invalid exercise type selected.")

print("\nReady to proceed to final bundling (Cell 10)")

##10.AUDIT ARTIFACTS

###10.1.OVERVIEW

**Cell 10: Bundling Your Audit Trail – Creating Complete Documentation**

**What This Cell Does:**

This final cell packages everything you've created during this session into a professional, downloadable audit trail. Think of it as closing out an engagement binder—you're gathering all working papers, creating an index, and preparing documentation that could be reviewed by a partner, regulator, or quality control team months or years from now. This isn't an afterthought; it's a demonstration that AI-assisted work can meet the same documentation standards as traditional professional services.

**The Audit README – Your Documentation Guide:**

The cell generates a comprehensive AUDIT_README.txt file that serves as the cover memo for your entire bundle. This document explains what each artifact is, why it exists, and how to use it. For someone unfamiliar with this notebook, the README provides step-by-step instructions: verify reproducibility by checking the config hash, review the redacted prompts carefully for any residual sensitive content, examine the risk register for flagged issues, validate that all deliverables include proper disclaimers, and obtain CPA sign-off before using anything in real engagements.

**Understanding the File Inventory:**

Before creating the zip file, the cell walks through your run directory and lists every single file that will be included. You'll see the manifest, the prompt log, the risk log, the README, and all deliverables from both the demos and your user exercise. This transparency is intentional—you should know exactly what's being bundled. The cell counts the files and displays their relative paths, giving you confidence that nothing is missing and nothing unexpected is included.

**The ZIP Bundle – Portable and Archivable:**

The cell uses Python's built-in compression tools to create a single ZIP file containing your entire run directory. The filename includes your unique run ID (timestamp plus config hash prefix), making it easy to distinguish this session from others if you run the notebook multiple times. The cell displays the ZIP file's size and location, and since it's saved in the /content/ directory, you can download it directly from Colab's file browser.

**The Final Checklist – Quality Assurance:**

Before declaring success, the cell runs through a six-item checklist verifying that all critical artifacts exist: manifest, prompt log, risk log, README, deliverables folder, and ZIP bundle. Each item shows a checkmark or X, giving you visual confirmation that your audit trail is complete and ready for review.

###10.2.CODE AND IMPLEMENTATION

In [22]:
# Cell 10: Bundle Artifacts + Audit Readme + Zip

print("=" * 70)
print("CELL 10: Final Artifact Bundling and Audit Trail")
print("=" * 70)

# Create AUDIT_README.txt
readme_text = f"""AUDIT TRAIL README
==================

Run ID: {RUN_ID}
Generated: {now_iso()}
Chapter: 2 (Level 2 Reasoners)
Model: {MODEL}

ARTIFACTS IN THIS BUNDLE
------------------------

1. run_manifest.json
   - Run ID, timestamp, model configuration
   - Config hash for reproducibility
   - Environment fingerprint (Python version, system info, library versions)
   - Full base configuration with control list

2. prompts_log.jsonl
   - JSONL format (one JSON object per line)
   - Redacted prompts and responses (PII removed via heuristics)
   - Cryptographic hashes (SHA256) for each prompt/response
   - Model name and temperature for each call
   - ⚠ CAUTION: Even though redacted, review carefully before sharing

3. risk_log.json
   - Risk register with entries for each deliverable
   - Risk types: confidentiality, independence, hallucination, missing_facts, qc, overreach, etc.
   - Severity levels (low/medium/high)
   - Linked to prompts via hashes for traceability

4. deliverables/ folder
   - Structured JSON outputs for each mini-case + user exercise
   - Human-readable .txt drafts with disclaimers
   - level2_minimum_standard.txt (quality standard checklist)

HOW TO REVIEW THIS AUDIT TRAIL
-------------------------------

STEP 1: Verify Reproducibility
- Check run_manifest.json for model, temperature, max_tokens
- Note config_hash; identical config = identical setup
- Compare environment fingerprint if reproducing in different environment

STEP 2: Review Prompts (with Caution)
- Open prompts_log.jsonl in a text editor
- Each line is a separate JSON object
- Prompts are redacted but may still contain sensitive context
- Use prompt_hash and response_hash to link to risk_log entries

STEP 3: Check Risk Register
- Open risk_log.json
- Review all flagged risks (especially "high" severity)
- Verify "Not verified" discipline was enforced
- Confirm overreach risks were caught

STEP 4: Validate Deliverables
- Review each *_output.json for strict schema compliance
- Check *_draft.txt files for:
  * Disclaimer present ("NOT ADVICE")
  * Facts vs assumptions separated
  * Open questions listed
  * Verification status = "Not verified" where appropriate
- Ensure no fabricated citations (check questions_to_verify)

STEP 5: CPA Review and Sign-Off
- All outputs are DRAFTS
- Engagement partner or manager must review
- Document review in engagement files
- Obtain sign-off before using in real engagements

LEVEL 2 BOUNDARY REMINDER
--------------------------

Level 2 reasoners provide STRUCTURED REASONING SUPPORT ONLY:
✓ Hypotheses, issue lists, research scaffolds
✓ Disconfirming-evidence questions (skepticism)
✓ Draft frameworks and checklists

Level 2 does NOT:
✗ Perform procedures or verify evidence
✗ Provide final conclusions or opinions
✗ Act as autonomous agents
✗ Replace professional judgment

Remember: Capability ↑ ⇒ Risk ↑ ⇒ Controls ↑

CONFIDENTIALITY WARNING
-----------------------

Even with redaction utilities:
- Do NOT share this bundle externally without scrubbing
- Review prompts_log.jsonl carefully (may contain context clues)
- Remove any client-specific information before archiving
- Follow your firm's data retention and disposal policies

SUPPORT
-------

For questions about this audit trail or Level 2 reasoners:
- Review level2_minimum_standard.txt in deliverables/
- Consult your firm's AI governance policies
- Contact: Alejandro Reynoso, Chief Scientist DEFI CAPITAL RESEARCH

This README is part of a research and training initiative.
Not for distribution without permission.
"""

readme_path = RUN_DIR / "AUDIT_README.txt"
readme_path.write_text(readme_text, encoding='utf-8')
print(f"✓ AUDIT_README.txt created: {readme_path}")

# List all files in run directory
print("\n" + "="*70)
print("FILES IN AUDIT TRAIL")
print("="*70)

all_files = []
for item in RUN_DIR.rglob('*'):
    if item.is_file():
        rel_path = item.relative_to(RUN_DIR)
        all_files.append(str(rel_path))
        print(f"  {rel_path}")

print(f"\nTotal files: {len(all_files)}")

# Create zip bundle
zip_filename = f"ai_audit_ch2_{RUN_ID}.zip"
zip_path = RUN_BASE_DIR / zip_filename

print("\n" + "="*70)
print("CREATING ZIP BUNDLE")
print("="*70)

import shutil
shutil.make_archive(
    str(RUN_BASE_DIR / f"ai_audit_ch2_{RUN_ID}"),
    'zip',
    RUN_DIR
)

print(f"✓ Zip bundle created: {zip_path}")
print(f"  Size: {zip_path.stat().st_size / 1024:.1f} KB")

# Final checklist
print("\n" + "="*70)
print("FINAL CHECKLIST")
print("="*70)

checklist = [
    ("run_manifest.json", (RUN_DIR / "run_manifest.json").exists()),
    ("prompts_log.jsonl", (RUN_DIR / "prompts_log.jsonl").exists()),
    ("risk_log.json", (RUN_DIR / "risk_log.json").exists()),
    ("AUDIT_README.txt", readme_path.exists()),
    ("deliverables/ folder", DELIVERABLES_DIR.exists()),
    ("Zip bundle", zip_path.exists())
]

for item, exists in checklist:
    status = "✓" if exists else "✗"
    print(f"  {status} {item}")

print("\n" + "="*70)
print("CHAPTER 2 NOTEBOOK COMPLETE")
print("="*70)

print(f"""
✓ All governance artifacts generated
✓ 4 mini-case demos completed
✓ User exercise ready
✓ Audit trail bundled

NEXT STEPS:
1. Download zip: {zip_path}
2. Review AUDIT_README.txt for detailed instructions
3. Examine deliverables/ folder for outputs
4. Obtain CPA review and sign-off before using outputs

Remember:
- These are DRAFTS, not final deliverables
- Facts ≠ Assumptions (validate all assumptions)
- "Not verified" discipline for authority-like statements
- Professional skepticism (disconfirming evidence)
- Engagement sign-off required

Thank you for using Level 2 Reasoners responsibly.
""")

print("\n🎓 TRAINING COMPLETE: You now understand Level 2 reasoning support for accounting & audit.")
print("📦 Download your audit trail and share with your team (after scrubbing sensitive data).")

CELL 10: Final Artifact Bundling and Audit Trail
✓ AUDIT_README.txt created: /content/ai_audit_ch2_runs/run_20260111_192247/AUDIT_README.txt

FILES IN AUDIT TRAIL
  AUDIT_README.txt
  prompts_log.jsonl
  run_manifest.json
  risk_log.json
  deliverables/Tax_ASC740_Research_Scaffold_Uncertain_Position_output.json
  deliverables/level2_minimum_standard.txt
  deliverables/Tax_ASC740_Research_Scaffold_Uncertain_Position_draft.txt
  deliverables/SOX_ICFR_Issue_Spotting_IT_Manual_Control_draft.txt
  deliverables/Teaching_Level2_Training_Rubric_Quiz_draft.txt
  deliverables/FS_Audit_Revenue_Variance_Hypotheses_output.json
  deliverables/SOX_ICFR_Issue_Spotting_IT_Manual_Control_output.json
  deliverables/Teaching_Level2_Training_Rubric_Quiz_output.json
  deliverables/FS_Audit_Revenue_Variance_Hypotheses_draft.txt

Total files: 13

CREATING ZIP BUNDLE
✓ Zip bundle created: /content/ai_audit_ch2_runs/ai_audit_ch2_20260111_192247_22c12306caf0.zip
  Size: 59.4 KB

FINAL CHECKLIST
  ✓ run_manifest.

##11.CONCLUSIONS

**Conclusion: From Casual Chat to Professional-Grade AI Assistance**

**The Complete Pipeline: Ten Steps to Trustworthy AI**

This notebook has walked you through a deliberate, methodical transformation of AI interaction from casual conversation to professionally defensible work product. The pipeline we've constructed represents a fundamental reimagining of how accounting and audit professionals should engage with generative AI systems.

The journey began with orientation and boundary-setting in Cell 1, establishing the core principle that Level 2 reasoners provide structured thinking support without crossing into autonomous decision-making or procedure execution. This wasn't mere disclaimer language—it defined the architectural constraints that every subsequent component respects. We then moved through technical foundation-building in Cells 2-3, where API connections and model parameters were explicitly configured and documented. The choice of low temperature (0.2) and the specification of model version weren't arbitrary technical details; they were governance decisions about consistency and reproducibility that get captured in the configuration hash.

Cells 4-5 constructed the governance infrastructure that operates invisibly but critically throughout every interaction. The utility functions for timestamping, hashing, logging, and redaction created a safety net that catches potential issues before they become problems. The environment fingerprint captured in Cell 4 ensures that someone reviewing this work months later can understand the exact technical context in which it was produced. The redaction utilities in Cell 5 acknowledged a hard truth: even well-intentioned professionals make mistakes with confidential data, and automated safeguards provide essential protection.

Cell 6 represented the intellectual heart of the system—the reasoner wrapper that transforms Claude from a general-purpose chatbot into a disciplined professional assistant. The strict JSON schema enforcement, the facts-versus-assumptions parsing, the automated risk flagging, and the "Not verified" discipline all converged in this single interface. Every subsequent AI interaction flowed through this wrapper, ensuring consistent application of governance controls regardless of who uses the notebook or what scenario they're analyzing.

The mini-cases in Cells 7-8 demonstrated the system under realistic professional conditions—revenue variance analysis, internal control assessment, uncertain tax position research, and training methodology development. These weren't sanitized textbook examples but authentic scenarios with incomplete information, ambiguous facts, and multiple defensible interpretations. Watching the system process these cases revealed how governance mechanisms operate under pressure: facts were explicitly separated from assumptions, open questions were generated systematically, authority references were flagged for verification, and risk registers captured concerns in structured format rather than vague unease.

Cell 9 transformed passive observation into active learning by letting you input your own scenario and experience the governed reasoning process firsthand. This hands-on component is crucial because understanding governance controls intellectually differs profoundly from experiencing them in practice. When you see your own scenario parsed into facts versus assumptions, when you receive disconfirming evidence questions that challenge your initial framing, when you get a risk register highlighting issues you hadn't consciously identified—that's when the value proposition becomes tangible rather than theoretical.

Finally, Cell 10 closed the loop by packaging the complete audit trail into a downloadable, reviewable, reproducible artifact bundle. The run manifest, prompts log, risk register, deliverables folder, and comprehensive README together constitute proof that professional standards were followed. This isn't bureaucratic overhead—it's the evidence base that makes AI-assisted work defensible in quality control reviews, regulatory examinations, or litigation contexts.

**The Pipeline's Core Innovation: Governance as Architecture, Not Afterthought**

What distinguishes this notebook from typical AI tutorials is the recognition that governance cannot be bolted onto AI systems after the fact. The confidentiality protections, verification discipline, audit trail generation, and risk flagging weren't added as optional enhancements—they were embedded into the core architecture from the first line of code. You cannot use this system without generating an audit trail. You cannot get AI output without explicit facts-versus-assumptions parsing. You cannot reference authorities without "Not verified" flags. The system makes ungoverned AI use harder than governed use, which inverts the usual dynamic where shortcuts are easier than best practices.

This architectural approach to governance has profound implications for professional services firms implementing AI. The usual path—deploy consumer chatbots, hope people use them responsibly, write policy documents nobody reads, deal with quality control failures reactively—is exactly backward. The correct sequence is: define governance requirements first, engineer systems that enforce those requirements by design, then deploy tools that make compliant usage the path of least resistance.

**The Essential Lesson: Control Enables Capability**

The fundamental insight this notebook should impart transcends the specific technical implementation or the particular governance mechanisms we've constructed. The essential lesson is this: rigorous controls don't constrain AI capability—they enable it. Without the governance infrastructure we've built, you cannot safely use AI for professional accounting and audit work, regardless of how impressive the underlying language model might be. With this infrastructure in place, AI becomes a legitimate tool for structuring professional judgment, generating hypotheses, identifying knowledge gaps, and documenting reasoning processes.

The casual AI user asks: "What's the answer to my question?" The professional AI user asks: "What facts have I provided? What assumptions am I making? What questions remain open? What risks exist in this analysis? How can I verify the output? How would I defend this work to a reviewer?" This notebook has shown you how to operationalize that professional mindset through deliberate system design. The controls we've implemented—facts versus assumptions, "Not verified" discipline, audit trails, risk registers, reproducibility mechanisms—aren't obstacles between you and AI productivity. They're the foundation that makes professional AI use possible at all.

As AI capabilities continue advancing at breakneck pace, the temptation will be to focus on what the newest models can do and to treat governance as a secondary concern that slows down innovation. Resist that temptation. The most important AI skill for professional services isn't prompt engineering or keeping up with model releases—it's the discipline to demand that powerful tools respect professional standards, maintain audit trails, acknowledge limitations explicitly, and support rather than replace human judgment. That discipline, more than any technical capability, will determine whether AI becomes a genuine asset to professional practice or a liability that undermines the credibility of the work we sign.